In [ ]:
#数据集链接
#https://www.kaggle.com/competitions/rsna-breast-cancer-detection
#https://www.kaggle.com/datasets/markwijkhuizen/keras-cv-attention-models
#https://www.kaggle.com/datasets/christofhenkel/nvidia-dali-nightly-cuda110-1230dev
#https://www.kaggle.com/datasets/christofhenkel/rsna-bc-pip-requirements
#https:训练完成的模型

In [ ]:
#!pip install -q timm==0.6.5 --no-index --find-links=/kaggle/input/rsna-bc-pip-requirements
#!pip install -q albumentations==1.2.1 --no-index --find-links=/kaggle/input/rsna-bc-pip-requirements
!pip install -q pylibjpeg-libjpeg==1.3.1 --no-index --find-links=/kaggle/input/rsna-bc-pip-requirements
!pip install -q pydicom==2.0.0 --no-index --find-links=/kaggle/input/rsna-bc-pip-requirements
!pip install -q python-gdcm==3.0.20 --no-index --find-links=/kaggle/input/rsna-bc-pip-requirements
!pip install -q dicomsdl==0.109.1 --no-index --find-links=/kaggle/input/rsna-bc-pip-requirements

!pip install -q /kaggle/input/nvidia-dali-nightly-cuda110-1230dev/nvidia_dali_nightly_cuda110-1.23.0.dev20230203-7187866-py3-none-manylinux2014_x86_64.whl

In [ ]:
# Install Keras CV Attention Model Pip Package for ConvNextV2 Models
!pip install --no-deps /kaggle/input/keras-cv-attention-models/keras_cv_attention_models-1.3.9-py3-none-any.whl

In [ ]:
%%writefile dcm_file_read.py    
#import timm
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
from copy import copy
import gc
import shutil 

import glob
from scipy.special import expit

#import albumentations as A
import cv2
cv2.setNumThreads(0)

import dicomsdl
import pydicom
from pydicom.filebase import DicomBytesIO

from os.path import join

from tqdm import tqdm

from joblib import Parallel, delayed
import multiprocessing as mp

from types import SimpleNamespace
from typing import Any, Dict

import torch
import torch.nn.functional as F
from torch import nn
from torch.nn.parameter import Parameter
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import GradScaler, autocast


import nvidia.dali.fn as fn
import nvidia.dali.types as types
from nvidia.dali import pipeline_def
from nvidia.dali.types import DALIDataType

#we need to patch DALI for Int16 support


from nvidia.dali.backend import TensorGPU, TensorListGPU
from nvidia.dali.pipeline import Pipeline
import nvidia.dali.ops as ops
from nvidia.dali import types
from nvidia.dali.plugin.base_iterator import _DaliBaseIterator
from nvidia.dali.plugin.base_iterator import LastBatchPolicy
import torch
import torch.utils.dlpack as torch_dlpack
import ctypes
import numpy as np
import torch.nn.functional as F
import pydicom

to_torch_type = {
    types.DALIDataType.FLOAT:   torch.float32,
    types.DALIDataType.FLOAT64: torch.float64,
    types.DALIDataType.FLOAT16: torch.float16,
    types.DALIDataType.UINT8:   torch.uint8,
    types.DALIDataType.INT8:    torch.int8,
    types.DALIDataType.UINT16:  torch.int16,
    types.DALIDataType.INT16:   torch.int16,
    types.DALIDataType.INT32:   torch.int32,
    types.DALIDataType.INT64:   torch.int64
}


def feed_ndarray(dali_tensor, arr, cuda_stream=None):
    """
    Copy contents of DALI tensor to PyTorch's Tensor.

    Parameters
    ----------
    `dali_tensor` : nvidia.dali.backend.TensorCPU or nvidia.dali.backend.TensorGPU
                    Tensor from which to copy
    `arr` : torch.Tensor
            Destination of the copy
    `cuda_stream` : torch.cuda.Stream, cudaStream_t or any value that can be cast to cudaStream_t.
                    CUDA stream to be used for the copy
                    (if not provided, an internal user stream will be selected)
                    In most cases, using pytorch's current stream is expected (for example,
                    if we are copying to a tensor allocated with torch.zeros(...))
    """
    dali_type = to_torch_type[dali_tensor.dtype]

    assert dali_type == arr.dtype, ("The element type of DALI Tensor/TensorList"
                                    " doesn't match the element type of the target PyTorch Tensor: "
                                    "{} vs {}".format(dali_type, arr.dtype))
    assert dali_tensor.shape() == list(arr.size()), \
        ("Shapes do not match: DALI tensor has size {0}, but PyTorch Tensor has size {1}".
            format(dali_tensor.shape(), list(arr.size())))
    cuda_stream = types._raw_cuda_stream(cuda_stream)

    # turn raw int to a c void pointer
    c_type_pointer = ctypes.c_void_p(arr.data_ptr())
    if isinstance(dali_tensor, (TensorGPU, TensorListGPU)):
        stream = None if cuda_stream is None else ctypes.c_void_p(cuda_stream)
        dali_tensor.copy_to_external(c_type_pointer, stream, non_blocking=True)
    else:
        dali_tensor.copy_to_external(c_type_pointer)
    return arr





# Params

COMP_FOLDER = '/kaggle/input/rsna-breast-cancer-detection/'
DATA_FOLDER = COMP_FOLDER + 'test_images/'

sample_submission = pd.read_csv(COMP_FOLDER + 'sample_submission.csv')

PUBLIC_RUN = len(sample_submission) == 2

N_CORES = mp.cpu_count()
MIXED_PRECISION = False
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

RAM_CHECK = False
DEBUG = False

test_df = pd.read_csv('/kaggle/input/rsna-breast-cancer-detection/test.csv')
test_df['cancer'] = 0 #dummy value


if PUBLIC_RUN is False:
    RAM_CHECK = False
    DEBUG = False

if RAM_CHECK is True:
    test_df = pd.read_csv('/kaggle/input/rsna-breast-cancer-detection/train.csv')
    patient_filter = list(sorted((set(test_df.patient_id.unique()))))[:8000]
    test_df = test_df[test_df.patient_id.isin(patient_filter)]
    DATA_FOLDER = DATA_FOLDER.replace('test','train')

if DEBUG is True:
    test_df = test_df.head(100)

test_df

print(f'Len df : {len(test_df)}')
test_df['patient_id'].nunique()

test_df["fns"] = test_df['patient_id'].astype(str) + '/' + test_df['image_id'].astype(str) + '.dcm'

def convert_dicom_to_jpg(file, save_folder=""):
    patient = file.split('/')[-2]
    image = file.split('/')[-1][:-4]
    dcmfile = pydicom.dcmread(file)

    if dcmfile.file_meta.TransferSyntaxUID == '1.2.840.10008.1.2.4.90':
        with open(file, 'rb') as fp:
            raw = DicomBytesIO(fp.read())
            ds = pydicom.dcmread(raw)
        offset = ds.PixelData.find(b"\x00\x00\x00\x0C")  #<---- the jpeg2000 header info we're looking for
        hackedbitstream = bytearray()
        hackedbitstream.extend(ds.PixelData[offset:])
        with open(save_folder + f"{patient}_{image}.jpg", "wb") as binary_file:
            binary_file.write(hackedbitstream)
            
    if dcmfile.file_meta.TransferSyntaxUID == '1.2.840.10008.1.2.4.70':
        with open(file, 'rb') as fp:
            raw = DicomBytesIO(fp.read())
            ds = pydicom.dcmread(raw)
        offset = ds.PixelData.find(b"\xff\xd8\xff\xe0")  #<---- the jpeg lossless header info we're looking for
        hackedbitstream = bytearray()
        hackedbitstream.extend(ds.PixelData[offset:])
        with open(save_folder + f"{patient}_{image}.jpg", "wb") as binary_file:
            binary_file.write(hackedbitstream)

            
@pipeline_def
def jpg_decode_pipeline(jpgfiles):
    jpegs, _ = fn.readers.file(files=jpgfiles)
    images = fn.experimental.decoders.image(jpegs, device='mixed', output_type=types.ANY_DATA, dtype=DALIDataType.UINT16)
    return images

def parse_window_element(elem):
    if type(elem)==list:
        return float(elem[0])
    if type(elem)==str:
        return float(elem)
    if type(elem)==float:
        return elem
    if type(elem)==pydicom.dataelem.DataElement:
        try:
            return float(elem[0])
        except:
            return float(elem.value)
    return None

def linear_window(data, center, width):
    lower, upper = center - width // 2, center + width // 2
    data = torch.clamp(data, min=lower, max=upper)
    return data 

def process_dicom(img, dicom):
    try:
        invert = getattr(dicom, "PhotometricInterpretation", None) == "MONOCHROME1"
    except:
        invert = False
        
    center = parse_window_element(dicom["WindowCenter"]) 
    width = parse_window_element(dicom["WindowWidth"])
        
    if (center is not None) & (width is not None):
        img = linear_window(img, center, width)

    img = (img - img.min()) / (img.max() - img.min())
    if invert:
        img = 1 - img
    return img

TARGET_HEIGHT = 1344
TARGET_WIDTH = 768
N_CHANNELS = 1
#INPUT_SHAPE = (TARGET_HEIGHT, TARGET_WIDTH, N_CHANNELS)
TARGET_HEIGHT_WIDTH_RATIO = TARGET_HEIGHT / TARGET_WIDTH

# Smooth vector used to smoothen sums/stds of axes
def smooth(l):
    # kernel size is 1% of vector
    kernel_size = int(len(l) * 0.01)
    kernel = np.ones(kernel_size) / kernel_size
    return np.convolve(l, kernel, mode='same')

# X Crop offset based on first column with sum below 5% of maximum column sums*std
def get_x_offset(image, max_col_sum_ratio_threshold=0.01, debug=None):
    # Image Dimensions
    H, W = image.shape[:2]
    # Percentual margin added to offset
    margin = int(image.shape[1] * 0.00)
    # Threshold values based on smoothed sum x std to capture varying intensity columns
    vv = smooth(image.sum(axis=0).squeeze()) * smooth(image.std(axis=0).squeeze())
    # Find maximum sum in first 75% of columns
    vv_argmax = vv[:int(image.shape[1] * 0.75)].argmax()
    # Threshold value
    vv_threshold = vv.max() * max_col_sum_ratio_threshold
    
    # Find first column after maximum column below threshold value
    for offset, v in enumerate(vv):
        # Start searching from vv_argmax
        if offset < vv_argmax:
            continue
        
        # Column below threshold value found
        if v < vv_threshold:
            offset = min(W, offset + margin)
            break
            

        
    return offset

# Y Crop offset based on first bottom and top rows with sum below 10% of maximum row sum*std
def get_y_offsets(image, max_row_sum_ratio_threshold=0.01, debug=None):
    # Image Dimensions
    H, W = image.shape[:2]
    # Margin to add to offsets
    margin = 0
    # Threshold values based on smoothed sum x std to capture varying intensity columns
    vv = smooth(image.sum(axis=1).squeeze()) * smooth(image.std(axis=1).squeeze())
    # Find maximum sum * std row in inter quartile rows
    vv_argmax = int(image.shape[0] * 0.25) + vv[int(image.shape[0] * 0.25):int(image.shape[0] * 0.75)].argmax()
    # Threshold value
    vv_threshold = vv.max() * max_row_sum_ratio_threshold
    # Default crop offsets
    offset_bottom = 0
    offset_top = H

    # Bottom offset, search from argmax to bottom
    for offset in reversed(range(0, vv_argmax)):
        v = vv[offset]
        if v < vv_threshold:
            offset_bottom = offset
            break
    

            
    # Top offset, search from argmax to top
    for offset in range(vv_argmax, H):
        v = vv[offset]
        if v < vv_threshold:
            offset_top = offset
            break
            

            
    return max(0, offset_bottom - margin), min(image.shape[0], offset_top + margin)

# Crop image and pad offsets to target image height/width ratio to preserve information
def crop(image, size=None, debug=False):
    # Image dimensions
    H, W = image.shape[:2]
    # Compute x/bottom/top offsets
    x_offset = get_x_offset(image, debug=debug)
    offset_bottom, offset_top = get_y_offsets(image[:,:x_offset], debug=debug)
    # Crop Height and Width
    h_crop = offset_top - offset_bottom
    w_crop = x_offset
    
    # Pad crop offsets to target aspect ratio
    if size is not None:
        # Height too large, pad x offset
        if (h_crop / w_crop) > TARGET_HEIGHT_WIDTH_RATIO:
            x_offset += int(h_crop / TARGET_HEIGHT_WIDTH_RATIO - w_crop)
        else:
            # Height too small, pad bottom/top offsets
            offset_bottom -= int(0.50 * (w_crop * TARGET_HEIGHT_WIDTH_RATIO - h_crop))
            offset_bottom_correction = max(0, -offset_bottom)
            offset_bottom += offset_bottom_correction

            offset_top += int(0.50 * (w_crop * TARGET_HEIGHT_WIDTH_RATIO - h_crop))
            offset_top += offset_bottom_correction
        
    # Crop Image
    image = image[offset_bottom:offset_top:,:x_offset]
        
    return image

cfg = SimpleNamespace(**{})
#cfg.img_size = 1024
#cfg.backbone = 'seresnext50_32x4d'
cfg.pretrained=False
cfg.in_channels = 1
cfg.classes = ['cancer']
cfg.batch_size = 8
cfg.data_folder = "/kaggle/tmp/output"
#cfg.val_aug = A.CenterCrop(always_apply=False, p=1.0, height=cfg.img_size, width=cfg.img_size)
cfg.device = DEVICE

#SAVE_SIZE = (768,1344)#int(cfg.img_size * 1.125)
SAVE_FOLDER = cfg.data_folder
os.makedirs(SAVE_FOLDER, exist_ok=True)
N_CHUNKS = len(test_df["fns"]) // 2000 if len(test_df["fns"]) > 2000 else 1
CHUNKS = [(len(test_df["fns"]) / N_CHUNKS * k, len(test_df["fns"]) / N_CHUNKS * (k + 1)) for k in range(N_CHUNKS)]
CHUNKS = np.array(CHUNKS).astype(int)
JPG_FOLDER = "/tmp/jpg/"



for ttt, chunk in enumerate(CHUNKS):
    print(f'chunk {ttt} of {len(CHUNKS)} chunks')
    os.makedirs(JPG_FOLDER, exist_ok=True)

    _ = Parallel(n_jobs=2)(
        delayed(convert_dicom_to_jpg)(f'{DATA_FOLDER}/{img}', save_folder=JPG_FOLDER)
        for img in test_df["fns"].tolist()[chunk[0]: chunk[1]]
    )
    
    jpgfiles = glob.glob(JPG_FOLDER + "*.jpg")


    pipe = jpg_decode_pipeline(jpgfiles, batch_size=1, num_threads=2, device_id=0)
    pipe.build()

    for i, f in enumerate(tqdm(jpgfiles)):
        
        patient, dicom_id = f.split('/')[-1][:-4].split('_')
        dicom = pydicom.dcmread(DATA_FOLDER + f"/{patient}/{dicom_id}.dcm")
        try:
            out = pipe.run()
            # Dali -> Torch
            img = out[0][0]
            img_torch = torch.empty(img.shape(), dtype=torch.int16, device="cuda")
            feed_ndarray(img, img_torch, cuda_stream=torch.cuda.current_stream(device=0))
            img = img_torch.float()
            #apply dicom preprocessing
            img = process_dicom(img, dicom)

            #resize the torch image
            #img = F.interpolate(img.view(1, 1, img.size(0), img.size(1)), (SAVE_SIZE, SAVE_SIZE), mode="bilinear")[0, 0]

            img = (img * 255).clip(0,255).to(torch.uint8).cpu().numpy()[:, :, 0]
            
            #out_file_name = SAVE_FOLDER + f"{patient}_{dicom_id}.png"
            #cv2.imwrite(out_file_name, img)
            h0, w0 = img.shape[:2]
            if img[:,int(-w0 * 0.10):].sum() > img[:,:int(w0 * 0.10)].sum():
                img = np.flip(img, axis=1)
            #img = crop(img, debug=False)  
            img = crop(img, size=(TARGET_WIDTH, TARGET_HEIGHT), debug=False)
            h, w = img.shape[:2]
            if (h / w) > TARGET_HEIGHT_WIDTH_RATIO:
                pad = int(h / TARGET_HEIGHT_WIDTH_RATIO - w)
                img = np.pad(img, [[0,0], [0, pad]])
                h, w = img.shape[:2]
            else:
                pad = int(0.50 * (w * TARGET_HEIGHT_WIDTH_RATIO - h))
                img = np.pad(img, [[pad, pad], [0,0]])
                h, w = img.shape[:2]
            # Resize
            img = cv2.resize(img, (TARGET_WIDTH, TARGET_HEIGHT), interpolation=cv2.INTER_AREA)
            cv2.imwrite(f'{SAVE_FOLDER}/{dicom_id}.jpg', img, [cv2.IMWRITE_JPEG_QUALITY, 100])
    
        except Exception as e:
            print(i, e)
            pipe = jpg_decode_pipeline(jpgfiles[i+1:], batch_size=1, num_threads=2, device_id=0)
            pipe.build()
            continue

    shutil.rmtree(JPG_FOLDER)
print(f'DALI Raw image load complete')

fns = glob.glob(f'{SAVE_FOLDER}/*.jpg')
n_saved = len(fns)
print(f'Image on disk count : {n_saved}')

gpu_processed_files = [fn.split('/')[-1].replace('jpg','dcm') for fn in fns]



to_process = [f for f in test_df["fns"].values if f.split('/')[-1] not in gpu_processed_files]
len(gpu_processed_files), len(to_process)


def process(f, save_folder=""):
    patient = f.split('/')[-2]
    dicom_id = f.split('/')[-1][:-4]
    
    dicom = dicomsdl.open(f)
    img = dicom.pixelData()
    img = torch.from_numpy(img)
    img = process_dicom(img, dicom)
    
    #img = F.interpolate(img.view(1, 1, img.size(0), img.size(1)), (SAVE_SIZE, SAVE_SIZE), mode="bilinear")[0, 0]

    img = (img * 255).clip(0,255).to(torch.uint8).cpu().numpy()
    
    
    #cv2.imwrite(out_file_name, img)
    h0, w0 = img.shape[:2]
    if img[:,int(-w0 * 0.10):].sum() > img[:,:int(w0 * 0.10)].sum():
        img = np.flip(img, axis=1)
    #img = crop(img, debug=False)  
    img = crop(img, size=(TARGET_WIDTH, TARGET_HEIGHT), debug=False)
    h, w = img.shape[:2]
    if (h / w) > TARGET_HEIGHT_WIDTH_RATIO:
        pad = int(h / TARGET_HEIGHT_WIDTH_RATIO - w)
        img = np.pad(img, [[0,0], [0, pad]])
        h, w = img.shape[:2]
    else:
        pad = int(0.50 * (w * TARGET_HEIGHT_WIDTH_RATIO - h))
        img = np.pad(img, [[pad, pad], [0,0]])
        h, w = img.shape[:2]
    # Resize
    img = cv2.resize(img, (TARGET_WIDTH, TARGET_HEIGHT), interpolation=cv2.INTER_AREA)
    cv2.imwrite(f'{SAVE_FOLDER}/{dicom_id}.jpg', img, [cv2.IMWRITE_JPEG_QUALITY, 100])
    out_file_name = f'{SAVE_FOLDER}/{dicom_id}.jpg'
    return out_file_name

cpu_processed_filenames = Parallel(n_jobs=2)(
    delayed(process)(f'{DATA_FOLDER}/{img}', save_folder=SAVE_FOLDER)
    for img in tqdm(to_process)
)
cpu_processed_filenames = [f for f in cpu_processed_filenames if f]
print(f'CPU Raw image load complete with {len(cpu_processed_filenames)} loaded')

gc.collect()
torch.cuda.empty_cache()

n_saved = len(glob.glob(f'{SAVE_FOLDER}/*.jpg'))
print(f'Image on disk count : {n_saved}')

assert n_saved == len(test_df)

In [ ]:
!python dcm_file_read.py

In [ ]:
%%writefile model1.py
from numba import cuda
import torch
cuda.select_device(0)
cuda.close()
cuda.select_device(0)
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import tensorflow as tf
from keras_cv_attention_models import convnext
import cv2
import glob
import os
import time
import gc
from scipy.stats import rankdata
TARGET_HEIGHT = 1344
TARGET_WIDTH = 768
N_CHANNELS = 3
INPUT_SHAPE = (TARGET_HEIGHT, TARGET_WIDTH, N_CHANNELS)
TARGET_HEIGHT_WIDTH_RATIO = TARGET_HEIGHT / TARGET_WIDTH

test = pd.read_csv('/kaggle/input/rsna-breast-cancer-detection/test.csv', usecols=['patient_id', 'image_id','laterality'])
test_paths = ('/kaggle/tmp/output/'+test['image_id'].map(str)+'.jpg').tolist()
def decode(path):
    file_bytes = tf.io.read_file(path)
    img = tf.image.decode_jpeg(file_bytes, channels=3)
    img = tf.cast(img, tf.float32) 
    return img
def build_dataset(slices, bsize=4, decode_fn=None):
    AUTO = tf.data.experimental.AUTOTUNE
    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.map(decode_fn, num_parallel_calls=AUTO)
    dset = dset.batch(bsize).prefetch(AUTO)
    return dset
dtest = build_dataset(test_paths, bsize=8,  decode_fn=decode)
def get_model():
    image = tf.keras.layers.Input(INPUT_SHAPE, name='image', dtype=tf.float32)
    image_norm = tf.keras.applications.imagenet_utils.preprocess_input(image, mode='torch')
    x = convnext.ConvNeXtV2Tiny(
        input_shape=(TARGET_HEIGHT, TARGET_WIDTH, 3),
        pretrained=None,
        num_classes=0,
    )(image_norm)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    model = tf.keras.models.Model(inputs=image, outputs=outputs)
    model.trainable = False
    model.compile()

    return model

tf.keras.backend.clear_session()
tf.config.optimizer.set_jit(True)
model_paths = ['/kaggle/input/rsna-trained-models/convnextv2_768/model_convext_exm4.h5',
              '/kaggle/input/rsna-trained-models/convnextv2_768/model_convext_exm3.h5',
              '/kaggle/input/rsna-trained-models/convnextv2_768/model_convext_exm2.h5',
              '/kaggle/input/rsna-trained-models/convnextv2_768/model_convext_exm1.h5',
              ]
with tf.device('/device:GPU:0'):
    models = []
    for p in model_paths:
        model = get_model()
        model.load_weights(p)
        models.append(model)

test['cancer'] = sum([model.predict(dtest, verbose=1) for model in models]) / len(models)
sub = test.groupby(['patient_id', 'laterality'])['cancer'].agg(['mean']).reset_index().rename(columns={'mean':'cancer'})
sub['prediction_id'] = sub['patient_id'].map(str)+'_'+sub['laterality'].map(str)
sub[['prediction_id', 'cancer']].to_csv('sub1.csv', index=False)

In [ ]:
%%writefile model2.py
from numba import cuda
import torch
cuda.select_device(0)
cuda.close()
cuda.select_device(0)
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import tensorflow as tf
from keras_cv_attention_models import convnext
import cv2
import glob
import os
import time
import gc
from scipy.stats import rankdata
TARGET_HEIGHT = 1344
TARGET_WIDTH = 768
N_CHANNELS = 3
INPUT_SHAPE = (TARGET_HEIGHT, TARGET_WIDTH, N_CHANNELS)
TARGET_HEIGHT_WIDTH_RATIO = TARGET_HEIGHT / TARGET_WIDTH

test = pd.read_csv('/kaggle/input/rsna-breast-cancer-detection/test.csv', usecols=['patient_id', 'image_id','laterality'])
test_paths = ('/kaggle/tmp/output/'+test['image_id'].map(str)+'.jpg').tolist()
def decode(path):
    file_bytes = tf.io.read_file(path)
    img = tf.image.decode_jpeg(file_bytes, channels=3)
    img = tf.cast(img, tf.float32) 
    return img
def build_dataset(slices, bsize=4, decode_fn=None):
    AUTO = tf.data.experimental.AUTOTUNE
    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.map(decode_fn, num_parallel_calls=AUTO)
    dset = dset.batch(bsize).prefetch(AUTO)
    return dset
dtest = build_dataset(test_paths, bsize=8,  decode_fn=decode)
def get_model():
    image = tf.keras.layers.Input(INPUT_SHAPE, name='image', dtype=tf.float32)
    image_norm = tf.keras.applications.imagenet_utils.preprocess_input(image, mode='torch')
    x = convnext.ConvNeXtV2Tiny(
        input_shape=(TARGET_HEIGHT, TARGET_WIDTH, 3),
        pretrained=None,
        num_classes=0,
    )(image_norm)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    model = tf.keras.models.Model(inputs=image, outputs=outputs)
    model.trainable = False
    model.compile()

    return model

tf.keras.backend.clear_session()
tf.config.optimizer.set_jit(True)
model_paths = [
    '/kaggle/input/rsna-trained-models/convnextv2_768/model_extra_all_f0.h5',
    '/kaggle/input/rsna-trained-models/convnextv2_768/model_extra_all_f1.h5',
    '/kaggle/input/rsna-trained-models/convnextv2_768/model_extra_all_f2.h5',
    '/kaggle/input/rsna-trained-models/convnextv2_768/model_extra_all_f4.h5',
              ]
with tf.device('/device:GPU:0'):
    models = []
    for p in model_paths:
        model = get_model()
        model.load_weights(p)
        models.append(model)

test['cancer'] = sum([model.predict(dtest, verbose=1) for model in models]) / len(models)
sub = test.groupby(['patient_id', 'laterality'])['cancer'].agg(['mean']).reset_index().rename(columns={'mean':'cancer'})
sub['prediction_id'] = sub['patient_id'].map(str)+'_'+sub['laterality'].map(str)
sub[['prediction_id', 'cancer']].to_csv('sub2.csv', index=False)

In [ ]:
!python model1.py

In [ ]:
!python model2.py

In [ ]:
import pandas as pd
import numpy as np
sub1 = pd.read_csv('sub1.csv').sort_values(['prediction_id']).reset_index(drop=True)
sub2 = pd.read_csv('sub2.csv').sort_values(['prediction_id']).reset_index(drop=True)

sub1['cancer'] = sub1['cancer']+sub2['cancer']

th = np.quantile(sub1['cancer'].values,0.98)#1.0-(1158+100)/53548
sub1['cancer'] = (sub1['cancer'].values > th).astype(int)

# Save submission as CSV
sub1.to_csv('submission.csv', index=False)